In [ ]:
# Importing Modules
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from sklearn import metrics
from google.colab import files
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Defining Plot
font_files = fm.findSystemFonts('.')

for font_file in font_files:
    fm.fontManager.addfont(font_file)
print(font_files)
plt.rc('font', family='UGent Panno Text')

plt.style.use('bmh')
plt.rcParams['font.size'] = 10
plt.rcParams['axes.labelsize'] = 10
plt.rcParams['axes.labelweight'] = 'bold'
plt.rcParams['axes.titlesize'] = 10
plt.rcParams['xtick.labelsize'] = 8
plt.rcParams['ytick.labelsize'] = 8
plt.rcParams['legend.fontsize'] = 10
plt.rcParams['figure.titlesize'] = 12

# Model definition
def CNN_OHE(tax, epochsize):
  print('Initiating training, validation and testing on {tax} using CNN with One Hot Encoded sequences')
  
  class my_precious_dataset(Dataset):
    def __init__(self, x, y):
        self.x = torch.tensor(x, dtype=torch.float32, device='cpu')
        self.y = torch.tensor(y, dtype=torch.long, device='cpu') # torch.long = 부호있는 정수형, torch.float = 32비트 소수형 
        self.length = self.x.shape[0]

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

    def __len__(self):
        return self.length


  # Load the existed Training & Validation & Testing Dataset
  base_path = '/content/drive/MyDrive/BachelorsProject/FinalModels/1AMBI'

  # These files are all in my google drive
  TrainX = np.load(f'{base_path}Train_X_OHE1A.npy')
  TrainY = np.load(f'{base_path}Train_Y_{tax}1A.npy')
  TestX = np.load(f'{base_path}Test_X_OHE1A.npy')
  TestY = np.load(f'{base_path}Test_Y_{tax}1A.npy')
  ValX = np.load(f'{base_path}Validation_X_OHE1A.npy')
  ValY = np.load(f'{base_path}Validation_Y_{tax}1A.npy')
  print('Training, test and validation datasets are loaded...')

  class Multi_Fungi_Classifier(nn.Module):
    def __init__(self):
        super(Multi_Fungi_Classifier, self).__init__()
        # First layer
        # Input shape = (100, 4, 700) 
        # 100: Batch size, 4: # of nucleotide, 700: Length of the sequences 
        self.conv1 = nn.Conv1d(in_channels=4, out_channels=8, kernel_size=3, stride=1, padding=1)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool1d(kernel_size=2, stride=2)
        # Input shape = (100, 8, 350)
        self.dropout = nn.Dropout(p=0.25)
        self.fc = nn.Linear(8 * 350, len(np.unique(TrainY))) 

        
    def forward(self, x):
        out = self.conv1(x)
        out = self.relu(out)
        out = self.pool(out)
        out = self.dropout(out)
        out = out.view(out.size(0), -1) # flatten
        out = self.fc(out)
        return out
  
    print('Model constructed...')

  model = Multi_Fungi_Classifier()

  batches = 200

  # Dataset
  trainset = my_precious_dataset(TrainX, TrainY)
  valset = my_precious_dataset(ValX, ValY)
  testset = my_precious_dataset(TestX, TestY)

  # Dataloader
  trainloader = DataLoader(trainset, batch_size=batches, shuffle=True)
  valloader = DataLoader(valset, batch_size=batches, shuffle=False)
  testloader = DataLoader(testset, batch_size=batches, shuffle=False)  

  # Model Training
  loss_fn = nn.CrossEntropyLoss()
  optimizer = optim.Adam(model.parameters(), lr=0.0001)

  # Store the training loss and the accuracy
  training_losses = []
  training_accuracy = []

  # Store the validation loss and the accuracy
  validation_losses = []
  validation_accuracy = []
      
  print('Training model...')
  epochs = epochsize
  for epoch in range(epochs):
      model.train()
      training_loss = 0.0
      correct = 0 # 예측값이 맞은 횟수
      total = 0

      # x_train is input data for the batch, y_train is the labels
      for i, (x_train, y_train) in enumerate(trainloader):
          input = x_train.transpose(1, 2)
          output = model(input)                        # Output shape: torch.Size([100, 16])
          loss = loss_fn(output, y_train)

          # Add L2 regularization
          l2_reg = torch.tensor(0.)
          for param in model.parameters():
              l2_reg += torch.norm(param)
          loss += 0.001 * l2_reg

          # Back propagation - gradients are calculated and the optimizer updates
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()

          # Calculate training loss and accuracy
          training_loss += loss.item() * input.size(0) # multiply the loss with the batch size (100)
          _, predicted = torch.max(output.data, 1)     # 0은 행 (세로), 1은 열 (가로) 마다 최댓값의 위치를 예측값으로 사용하겠다는 의미! * 원래 torch.max 는 최댓값, 최댓값의 위치를 산출해주는데, 최댓값은 필요없으므로 _ 로 저장 x
          total += y_train.size(0)                     # Total number of predictions 
          correct += (predicted == y_train).sum().item() # 예측값과 라벨이 맞을때의 개수 * item()이 없으면 tensor(64) 라고 나옴. 

      # Print training statistics
      epoch_loss = training_loss / len(trainloader.dataset) # Training loss is calculated by dividing the cumulative loss by the total number of data points in the training dataset
      epoch_acc = 100. * correct / total   
      
      # Store the training loss and the accuracy
      training_losses.append(epoch_loss)
      training_accuracy.append(epoch_acc)
      
      # Validation
      model.eval()
      validation_loss = 0.0
      correct = 0
      total = 0
    
      with torch.no_grad():
        for i, (x_val, y_val) in enumerate(valloader):
            input = x_val.transpose(1, 2)
            output = model(input)
            loss = loss_fn(output, y_val)
            
            # Calculate validation loss and accuracy
            validation_loss += loss.item() * input.size(0)
            _, predicted = torch.max(output.data, 1)
            total += y_val.size(0)
            correct += (predicted == y_val).sum().item()

      # Calculate validation loss and accuracy
      epoch_val_loss = validation_loss / len(valloader.dataset)
      epoch_val_acc = 100. * correct / total

      # Store the validation loss and the accuracy
      validation_losses.append(epoch_val_loss)
      validation_accuracy.append(epoch_val_acc)

      print(f'Epoch [{epoch+1}/{epochs}], Training Loss: {epoch_loss:.4f}, Validation Loss: {epoch_val_loss:.4f}, Training Accuracy: {epoch_acc:.2f}%, Validation Accuracy: {epoch_val_acc:.2f}%')

  # Set the model to evaluation mode
  model.eval()

  with torch.no_grad():
    y_prediction = []
    y_true = []

    for x_test, y_test in testloader:
      test_input = x_test.transpose(1, 2)
      test_output = model(test_input)
      y_prediction += torch.argmax(test_output, dim=1).tolist()
      y_true += y_test.tolist()
    print(metrics.classification_report(y_true, y_prediction, digits=3))     

  plt.plot(training_losses, label='Training', color='#1E64C8', linewidth=1)
  plt.plot(validation_losses, label='Validation', color='black', linewidth=1)
  plt.title(f'Training and Validation Loss of the CNN on {tax} level with OHE')
  plt.xlabel('Epoch')
  plt.ylabel('Loss (in %)')
  plt.legend()
  plt.savefig(f'CNNOHE{tax}{kmer}Loss1A.svg')
  files.download(f'CNNOHE{tax}{kmer}Loss1A.svg') 
  plt.show()

  plt.plot(training_accuracies, label='Training', color='#1E64C8', linewidth=1)
  plt.plot(validation_accuracies, label='Validation', color='black', linewidth=1)
  plt.title(f'Training and Validation Accuracy of the CNN on {tax} level with {kmer}')
  plt.xlabel('Epoch')
  plt.ylabel('Accuracy (in %)')
  plt.legend()
  plt.savefig(f'CNNOHE{tax}Accuracy1A.svg')
  files.download(f'CNNOHE{tax}Accuracy1A.svg') 
  plt.show()
  print(f'Training, validation and testing on {tax} level with OHE is completed.') 

We then run the model on all taxonomic levels with 3mer, 4mer and 5mer.

In [ ]:
#Running the Model on 3mer, 4mer, 5mer
# 3 mer
CNNK('3mer', 'phylum', 100)
CNNK('3mer', 'class', 100)
CNNK('3mer', 'order', 100)
CNNK('3mer', 'family', 100)
CNNK('3mer', 'genus', 100)

# 4 mer
CNNK('4mer', 'phylum', 100)
CNNK('4mer', 'class', 100)
CNNK('4mer', 'order', 100)
CNNK('4mer', 'family', 100)
CNNK('4mer', 'genus', 100)

# 5 mer
CNNK('5mer', 'phylum', 100)
CNNK('5mer', 'class', 100)
CNNK('5mer', 'order', 100)
CNNK('5mer', 'family', 100)
CNNK('5mer', 'genus', 100)